In [6]:
from datasets import load_dataset
DATASET_NAME='arc_hard'
dataset = load_dataset("tau/commonsense_qa", "", split=["train", "validation"])

In [7]:
dataset[1][0]

{'id': '1afa02df02c908a558b4036e80242fac',
 'question': 'A revolving door is convenient for two direction travel, but it also serves as a security measure at a what?',
 'question_concept': 'revolving door',
 'choices': {'label': ['A', 'B', 'C', 'D', 'E'],
  'text': ['bank', 'library', 'department store', 'mall', 'new york']},
 'answerKey': 'A'}

In [8]:
def get_prompt(json_line, has_choice=False):
    try:
        question = json_line["question"]
        choices = json_line["choices"]
        choice_texts = choices["text"]
        perms=list(range(len(choice_texts)))
        choice_texts = [choice_texts[perms[i]] for i in range(len(choice_texts))]
        candidates = " ".join(
            [
                f"({label if has_choice else ' '}) {text}"
                for text, label in zip(choice_texts, choices["label"])
            ]
        ).replace("\n", " ")
        answer_key = json_line["answerKey"][0]
        answer_key_idx = ord(answer_key) - (ord("A") if answer_key in 'ABCDE' else ord("1"))
        answer_text = choices["text"][answer_key_idx]
        fact=f"{json_line['fact1']}. " if 'fact1' in json_line else ''
        prompt = f"{fact}{question} \\n {candidates}"
        return prompt, answer_text
    except:
        print(answer_key)

In [9]:
from tqdm import tqdm
import pickle
container=[get_prompt(x, has_choice=False) for x in tqdm(dataset[0])]
pickle.dump(container, open(f"{DATASET_NAME}_train.pkl", "wb"))
pickle.dump([get_prompt(x, has_choice=False) for x in tqdm(dataset[1])], open(f"{DATASET_NAME}_test.pkl", "wb"))

100%|██████████| 1221/1221 [00:00<00:00, 7928.54it/s]
